# Getting Started with PyTorch Part 1: Understanding how Automatic Differentiation works
## Prerequisites

Before we begin, I must point out that you should have at least the basic idea about:
- Concepts related to training of neural networks, particularly backpropagation and gradient descent.
- Applying the chain rule to compute derivatives.
- How classes work in Python. (Or a general idea about Object Oriented Programming)

In case, you’re missing any of the above, I’ve provided links at the end of the article to guide you.


So, it’s time to get started with PyTorch. This is the first in a series of tutorials on PyTorch.

This is the part 1 where I’ll describe the basic building blocks, and Autograd.

**_NOTE:_** _An important thing to notice is that the tutorial is made for PyTorch 0.3 and lower versions. The latest version on offer is 0.4. I’ve decided to stick with 0.3 because as of now, 0.3 is the version that is shipped in Conda and pip channels. Also, most of PyTorch code that is used in open source hasn’t been updated to incorporate some of the changes proposed in 0.4. I, however, will point out at certain places where things differ in 0.3 and 0.4._

## Building Block #1 : Tensors

If you’ve ever done machine learning in python, you’ve probably come across NumPy. The reason why we use Numpy is because it’s much faster than Python lists at doing matrix ops. Why? Because it does most of the heavy lifting in C.

But, in case of training deep neural networks, NumPy arrays simply don’t cut it. I’m too lazy to do the actual calculations here (google for “FLOPS in one iteration of ResNet to get an idea), but code utilising NumPy arrays alone would take months to train some of the state of the art networks.

This is where __Tensors__ come into play. PyTorch provides us with a data structure called a Tensor, which is very similar to NumPy’s ndarray. But unlike the latter, __tensors can tap into the resources of a GPU to significantly speed up matrix operations.__

Here is how you make a Tensor.



In [88]:
import torch
import numpy as np

arr = np.random.randn(3,5)
arr

array([[-0.98907602,  0.37633641, -0.52628643,  0.56717601,  1.6970492 ],
       [-1.02104835, -0.39138278, -0.35093669, -1.2368533 ,  0.28203838],
       [-0.45269895,  0.13519031, -0.79994334,  0.30100095,  1.6255636 ]])

In [89]:
tens = torch.from_numpy(arr)
tens

tensor([[-0.9891,  0.3763, -0.5263,  0.5672,  1.6970],
        [-1.0210, -0.3914, -0.3509, -1.2369,  0.2820],
        [-0.4527,  0.1352, -0.7999,  0.3010,  1.6256]], dtype=torch.float64)

In [90]:
another_tensor = torch.LongTensor([[2,4],[5,6]])
another_tensor

tensor([[2, 4],
        [5, 6]])

In [91]:
random_tensor = torch.randn((4,3)) # torch.randn(4,3)
random_tensor

tensor([[-1.2457, -0.8909, -0.3537],
        [ 1.0850, -0.0927, -0.9277],
        [ 0.1946,  0.7974,  0.2164],
        [ 0.4712,  0.6790,  0.2056]])

## Building Block #2 : Computation Graph
Now, we are at the business side of things. When a neural network is trained, we need to compute gradients of the loss function, with respect to every weight and bias, and then update these weights using gradient descent.

With neural networks hitting billions of weights, doing the above step efficiently can make or break the feasibility of training.

### Building Block #2.1: Computation Graphs
Computation graphs lie at the heart of the way modern deep learning networks work, and PyTorch is no exception. Let us first get the hang of what they are.

Suppose, your model is described like this:

$$ 
\begin{aligned}
& b = w_1 \times a\\
& c = w_2 \times a \\
& d = (w_3 \times b) \times (w_4 \times c)\\
& L = f(d)
\end{aligned}
$$

If I were to actually draw the computation graph, it would probably look like this.

<figure class="image">
  <img src="Computation Graph for our Model.png" alt=""
  <figcaption>Computation Graph for our Model</figcaption>
</figure>

**NOW**, you must note, that the above figure is not entirely an accurate representation of how the graph is represented under the hood by PyTorch. However, for now, it’s enough to drive our point home.

Why should we create such a graph when we can sequentially execute the operations required to compute the output?

Imagine, what were to happen, if you didn’t merely have to calculate the output but also train the network. You’ll have to compute the gradients for all the weights labelled by purple nodes. That would require you to figure your way around chain rule, and then update the weights.


**The computation graph is simply a data structure that allows you to efficiently apply the chain rule to compute gradients for all of your parameters.**

<figure class="image">
  <img src="Applying the chain rule using computation graphs.png" alt=""
  <figcaption>Applying the chain rule using computation graphs</figcaption>
</figure>

Here are a couple of things to notice. First, that the directions of the arrows are now reversed in the graph. That’s because we are backpropagating, and arrows marks the flow of gradients backwards.

Second, for the sake of these example, you can think of the gradients I have written as edge weights. Notice, these gradients don’t require chain rule to be computed.

Now, in order to compute the gradient of any node, _say, L, with respect of any other node, say c ( dL / dc)_ all we have to do is:

1. Trace the path from L to c. This would be L → d → c.
2. Multiply all the edge weights as you traverse along this path. The quantity you end up with is: _( dL / dd ) * ( dd / dc ) = ( dL / dc)_
3. If there are multiple paths, add their results. For example in case of dL/da, we have two paths. *L → d → c → a* and *L → d → b→ a*. We add their contributions to get the gradient of *L* w.r.t. *a*

_[( dL / dd ) * ( dd / dc ) * ( dc / da )] + [( dL / dd ) * ( dd / db ) * ( db / da )]_

In principle, one could start at *L*, and start traversing the graph backwards, calculating gradients for every node that comes along the way.


## Building Block #3 : Variables and Autograd
PyTorch accomplishes what we described above using the Autograd package.

Now, there are basically three important things to understand about how Autograd works.

### Building Block #3.1 : Variable
The Variable, just like a Tensor is a class that is used to hold data. It differs, however, in the way it’s meant to be used. **Variables are specifically tailored to hold values which change during training of a neural network, i.e. the learnable paramaters of our network**. Tensors on the other hand are used to store values that are not to be learned. For example, a Tensor maybe used to store the values of the loss generated by each example.

In [92]:
from torch.autograd import Variable
var_ex = Variable(torch.randn((4,3))) # creating a Variable

A _Variable_ class wraps a tensor. You can access this tensor by calling **.data** attribute of a Variable.

The _Variable_ also stores the gradient of a scalar quantity (say, loss) with respect to the parameter it holds. This gradient can be accessed by calling the __.grad__ attribute. This is basically the gradient computed up to this particular node, and the gradient of the every subsequent node, can be computed by multiplying the _edge weight_ with the gradient computed at the node just before it.

The third attribute a Variable holds is a __grad_fn__, a Function object which created the variable.


<figure class="image">
  <img src="Variable_and_function.png" alt=""
  <figcaption>Variable and function</figcaption>
</figure>

**_NOTE:_** *PyTorch 0.4 merges the Variable and Tensor class into one, and Tensor can be made into a “Variable” by a switch rather than instantiating a new object. But since, we’re doing v 0.3 in this tutorial, we’ll go ahead.*

### Building Block #3.2 : Function
Did I say _Function_ above? It is basically an abstraction for, well, a function. Something that takes an input, and returns an output. For example, if we have two variables, _a_ and _b_, then if,

$c = a + b$

Then _c_ is a new variable, and it’s _grad_fn_ is something called _AddBackward_ (PyTorch’s built-in function for adding two variables), the function which took _a_ and _b_ as input, and created _c_.

Then, you may ask, why is a need for an entire new class, when python does provide a way to define function?


While training neural networks, there are two steps: the forward pass, and the backward pass. Normally, if you were to implement it using python functions, you will have to define two functions. One, to compute the output during forward pass, and another, to compute the gradient to be propagated.

**PyTorch abstracts the need to write two separate functions (for forward, and for backward pass), into two member of functions of a single class called _torch.autograd.Function_.**

PyTorch combines _Variables_ and _Functions_ to create a computation graph.

### Building Block #3.3 : Autograd
Let us now dig into how PyTorch creates a computation graph. First, we define our variables.

In [93]:
from torch import FloatTensor
from torch.autograd import Variable

# Define the leaf nodes
a = Variable(FloatTensor([4]) , requires_grad=True) # not necessary to , requires_grad=True

weights = [Variable(FloatTensor([i]), requires_grad=True) for i in (2,5,9,7)]

# unpack the weights for nicer assignment
w1, w2, w3, w4 = weights

b = w1*a
c = w2*a
d = w3*b + w4*c
# d.retain_grad()
L = (10-d)

L.backward()

for index, weight in enumerate(weights, start=1):
    gradient, *_ = weight.grad.data
    print(f"Gradient of w{index} w.r.t. L: {gradient}")

print(w1.grad.data)
print(a, a.grad.data) # a is leaf node, so can calculaet gradient
print(d, d.grad) # d isn't leaf node, so cannot calculate gradient

Gradient of w1 w.r.t. L: -36.0
Gradient of w2 w.r.t. L: -28.0
Gradient of w3 w.r.t. L: -8.0
Gradient of w4 w.r.t. L: -20.0
tensor([-36.])
tensor([4.], requires_grad=True) tensor([-53.])
tensor([212.], grad_fn=<AddBackward0>) None


Now, let’s dissect what the hell just happened here. If you look at the source code, here is how things go.

- **Define the leaf variables of the graph (Lines 5–9)**. We start by defining a bunch of “variables” (Normal, python usage of language, not pytorch Variables). If you notice, the values we defined are the leaf nodes in the our computation graph. It only makes sense that we have to define them since these nodes aren’t result of any computation. At this point, these guys now occupy memory in our Python namespace. Means, they are hundred percent real. We **must** set the **requires_grad** attribute to True, otherwise, these Variables won’t be included in the computation graph, and no gradients would be computed for them (and other variables, that depend on these particular variables for gradient flow).

- **Create the graph (Lines 12–15)**. Till now, there is nothing such as computation graph in our memory. Only the leaf nodes, but as soon as you write lines 12–15, a graph is being generated **ON THE FLY. REALLY IMPORTANT TO NAIL THIS DETAIL. ON THE FLY.** When you write _b =w1*a_, it’s when the graph creation kicks in, and continues until line 15. This is precisely the forward pass of our model, when the output is being calculated from inputs. The forward function of each variable may cache some input values to be used while computing the gradient on the backward pass. (For example, if our forward function computes _W*x_, then _d(W*x)/d(W)_ is _x_, the input that needs to be cached)

- Now, the reason I told you the graph I drew earlier wasn’t exactly accurate? Because when PyTorch makes a graph, it’s not the Variable objects that are the nodes of the graph. It’s a _Function_ object, precisely, the _grad_fn_ of each Variable that forms the nodes of the graph. So, the PyTorch graph would look like.

<figure class="image">
  <img src="Each Function is a node in the PyTorch computation graph.png" alt=""
  <figcaption>Each Function is a node in the PyTorch computation graph</figcaption>
</figure>

- I’ve represented the leaf nodes, by their names, but they too have their *grad_fn*’s (which return a None value . It makes sense, as you can’t backpropagate beyond leaf nodes). The rest of nodes are now replaced by their *grad_fn*’s. We see that the single node *d* is replaced by three Functions, two multiplications, and an addition, while loss, is replaced by a minus Function.

- Compute the Gradients (Line 18). We now compute the gradients by calling the *.backward()* function on *L*. What exactly is going on here? First, the gradient at *L*, is simply 1 *(dL / dL)*. **Then, we invoke it’s backward function, which basically has a job of computing the gradients of the output of the Function object, w.r.t to the inputs of the Function object**. Here, *L* is the result of 10 - *d*, which means, backwards function will compute the gradient _(dL/dd)_ as -1.

- Now, this computed gradient is multiplied by the accumulated gradient (Stored in the grad attribute of the Variable corresponding to the current node, which is *dL/dL = 1* in our case), and then sent to input node, to be stored in the **grad attribute of the Variable corresponding to input node**. Technically, what we have done is apply the chain rule *(dL/dL) * (dL/dd) = dL/dd*.

- Now, let us understand how gradient is propagated for the Variable *d*. *d* is calculated from it’s inputs *(w3, w4, b, c)*. In our graph, it consists of 3 nodes, 2 multiplications and 1 addition.

- First, the function *AddBackward* (representing addition operation of node d in our graph) computes the gradient of it’s output _(w3*b + w4*c)_ w.r.t it’s inputs (_w3*b_ and _w4*c_ ), which is (1 for both). Now, these local gradients are multiplied by accumulated gradient (*dL/dd x 1 = -1* for both), and the results are saved in the *grad* attribute of the respective input nodes.

- Then, the Function _MulBackward_ (representing multiplication operation of _w4*c_  ) computes the gradient of it’s input output w.r.t to it’s inputs ( _w4_ and _c_ ) as ( _c_ and _w4_ ) respectively. The local gradients are multiplied by accumulated gradient ( _dL/d(w4*c) = -1_ ). The resultant value ( _-1 x c_ and _-1 x w4_ ) is then stored in grad attribute of Variables _w4_ and _c_ respectively.

- Gradients for all the nodes are computed in a similar fashion.

- The gradient of _L_ w.r.t any node can be accessed by calling _.grad_ on the Variable corresponding to that node, _given it’s a leaf node_ (PyTorch’s default behavior doesn’t allow you to access gradients of non-leaf nodes. More on that in a while). Now that we have got our gradients, we can update our weights using SGD or whatever optimization algorithm you like.

 

In [64]:
learning_rate = 0.002

print(w1, w1.grad)
w1 =w1 - learning_rate*w1.grad   # update the wieghts using GD

print(w1, w1.grad)

tensor([2.], requires_grad=True) tensor([-36.])
tensor([2.0720], grad_fn=<SubBackward0>) None


and so forth.


## Some Nifty Details of Autograd
So, didn’t I tell you you can’t access the grad attribute of non-leaf Variables. Yeah, that’s the default behavior. You can override it by calling .retain_grad() on the Variable just after defining it and then you’d be able to access it’s grad attribute. But really, what the heck is going on under the wraps.

### Dynamic Computation Graphs
PyTorch creates something called a **Dynamic Computation Graph**, which means that the graph is generated on the fly. **Until the forward function of a Variable is called, there exists no node for the Variable (it’s _grad_fn_ ) in the graph**. The graph is created as a result of forward function of many Variables being invoked. Only then, the buffers are allocated for the graph and intermediate values (used for computing gradients later). When you call backward(), as the gradients are computed, these buffers are essentially freed, and the graph is destroyed. You can try calling backward() more than once on a graph, and you’ll see PyTorch will give you an error. This is because the graph gets destroyed the first time backward() is called and hence, there’s no graph to call backward upon the second time.

If you call _forward_ again, an entirely new graph is generated. With new memory allocated to it.

__By default, only the gradients ( _grad_ attribute) for leaf nodes are saved, and the gradients for non-leaf nodes are destroyed__. But this behavior can be changed as described above.

This is in contrast to the __Static Computation Graphs__, used by TensorFlow where the graph is declared _before_ running the program. The dynamic graph paradigm allows you to make changes to your network architecture during runtime, as a graph is created only when a piece of code is run. This means a graph may be redefined during the lifetime for a program. This, however, is not possible with static graphs where graphs are created before running the program, and merely executed later. Dynamic graphs also make debugging way easier as the source of error is easily traceable.

## Some Tricks of Trade
### requires_grad
This is an attribute of the _Variable_ class. By default, it’s False. It comes handy when you have to freeze some layers, and stop them from updating parameters while training. You can simply set the _requires_grad_ to False, and these _Variables_ won’t be included in the computation graph. Thus, no gradient would be propagated to them, or to those layers which depend upon these layers for gradient flow. _requires_grad_, _when set to True is contagious_, meaning even if one operand of an operation has requires_grad set to True, so will the result.
<figure class="image">
  <img src="freeze_variable.png" alt=""
  <figcaption>b is not included in the graph. No gradient is backpropagated through b now. a only gets gradients from c now. Even if w1 has requires_grad = True, there is no way it can receive gradients.
</figcaption>

</figure>

### volatile
This again is a attribute of a _Variable_ class, which causes a _Variable_ to be excluded from the computation graph when it is set to True. It might seem quite similar to _requires_grad_, given it’s also __contagious when _set_ True__. But it has a _higher_ precedence than _requires_grad_. __A _variable_ with _requires_grad_ equals to True and volatile equals to True, would not be included in the computation graph.__

You might think, what’s the need of having another switch to override _requires_grad_, when we can simply set _requires_grad_ to False? Let me digress for a while.

_Not creating_ a graph is extremely useful when we are doing inference, and _don’t need gradients_. First, _overhead_ to create a computation graph is eliminated, and the speed is boosted. Second, if we create a graph, and since there is no backward being called afterwords, the buffers used to cache values are never freed and may lead to you running out of memory.

Generally, we have many layers in the a neural network, for which we might have set _requires_grad_ to True while training. To prevent a graph from being made at inference, we can do either of two things. Set _requires_grad_ False on all the layers (maybe, 152 of them?). Or, set _volatile_ True only on the input, and we’re assured no resultant operation will result in a graph being made. Your choice.
<figure class="image">
  <img src="volatile.png" alt=""
  <figcaption>No graph is created for b or any node that depends on b.</figcaption>
</figure>

*__NOTE:__ PyTorch 0.4 has no volatile argument for a combined Tensor/Variable class. Instead, the inference code should be put in a torch.no_grad() context manager.*


In [87]:
with torch.no_grad():
    pass
    #-----  your inference code goes here ----


## Conclusion
So, that was _Autograd_ for you. Understanding how Autograd works can save you a lot of headache when you’re stuck somewhere, or dealing with errors when you’re starting out. Thanks for reading so far. I intend to write more tutorials on PyTorch, dealing with how to use inbuilt functions to quickly create complex architectures (or, maybe not so quickly, but faster than coding block by block). So, stay tuned!

## Further Reading
1. [Understanding Backpropagation](http://neuralnetworksanddeeplearning.com/chap2.html)
2. [Understanding the Chain Rule](https://www.youtube.com/watch?v=MKWBx78L7Qg)
3. Classes in Python [Part 1](https://www.hackerearth.com/practice/python/object-oriented-programming/classes-and-objects-i/tutorial/) and [Part 2](https://www.hackerearth.com/practice/python/object-oriented-programming/classes-and-objects-ii-inheritance-and-composition/tutorial/)
4. [PyTorch’s Official Tutorial](http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
